In [2]:
import sys
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
import pickle
import re
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer
from transformers import EvalPrediction
import torch
import datasets
from datasets import Dataset

import warnings
import transformers

warnings.filterwarnings('ignore')
print(transformers.__version__)
torch.set_printoptions(profile="full")

4.27.2


In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# !{sys.executable} -m pip install datasets

In [ ]:
# dataset_path = 'topics'
# topic_names = ['text']
# for x in os.listdir(dataset_path):
#     topic_names.append(x)
# topic_names

In [ ]:
# dataset = pd.DataFrame(columns=topic_names)
# dataset

,text,alone,america,angel,anger,animal,baby,beach,beautiful,beauty,...,trust,truth,war,warning,water,weather,wedding,winter,work,world


In [ ]:
# for topic in tqdm(os.listdir(dataset_path)):
#     for file in os.listdir(f'{dataset_path}/{topic}'):
#         with open(f'{dataset_path}/{topic}/{file}', encoding = 'utf-8') as f:
#             str = f.read()
#             index = dataset.index[dataset['text'] == str]
#             # print(index.size)
#             if index.size != 0:  #  если текст есть в датасете
#                 # print('not None')
#                 dataset[topic][index] = 1
#             else:   # если текст отсутствует в датасете
                
#                 dataset = dataset.append({'text': str, topic: 1}, ignore_index=True)
#                 # print(file)

100%|██████████| 144/144 [2:31:58<00:00, 63.32s/it] 


In [ ]:
# dataset

,text,alone,america,angel,anger,animal,baby,beach,beautiful,beauty,...,trust,truth,war,warning,water,weather,wedding,winter,work,world
0,loneliness\nhis childhood\nwarms him up\nlonel...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Sit in a chair\nalone\nPeople I thought were\n...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Tell me why it has to be this way\nwhy I must ...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"A debtor to mercy alone, of covenant mercy I s...",1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,A year has been passed since I left home\nAgai...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13732,"Wake not for the world-heard thunder,\nNor the...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
13733,"We may roam through this world, like a child a...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
13734,When you were born in this world\nEveryone lau...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
13735,"WORLD, take good notice, silver stars fading,\...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


In [ ]:
# dataset_fillna = dataset.fillna(0)

In [ ]:
# dataset

,text,passion,daughter,weather,animal,sea,innocence,change,angel,brother,...,freedom,dark,hate,music,warning,teacher,peace,chicago,butterfly,murder
0,"Your passion is permanently etched on my soul,...",1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"When man had ceased to utter his lament, A god...",1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"O Gentle Love, ungentle for thy deed, Thou mak...",1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,PASSION TOUCHED HER LIPS Passion touched her l...,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,You eclipse me & I have stained the Sun with b...,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13731,Violence is nobody s leaven. Murder will not g...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
13732,St. Patrick s Church was sparsely filled For F...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
13733,No one sleeps more beautifully than you. But I...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
13734,"Pastor in the pulpit, the Sanctuary, supposedl...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [ ]:
# with open(r'dataset_fillna.pkl', 'wb') as f:
#     pickle.dump(dataset_fillna, f)

Загрузка датасета

In [ ]:
# with open(r'dataset_fillna.pkl', 'rb') as f:
#     dataset = pickle.load(f)

Обработка датасета

In [ ]:
# EXCLUDE_SYMBOLS_STR = u''.join(['№', '«', 'ђ', '°', '±', 'ћ', '‰', '»', 'ѓ', 'µ', '·', 'ґ', 'њ', 'ї', 'џ', 'є', '‹',
#                                 '‡', '†', '¶', 'ќ', '€', '“', 'ў', '§', '„', '”', '\ufeff', 'љ', '›', '•', '—', 
#                                 '\x7f', '\xad', '¤', '\xa0', '\u200e', 'Š', 'ō', 'ä', '™', '×', '\'', '~'])

# regex_symb = re.compile('[%s]' % re.escape(EXCLUDE_SYMBOLS_STR))

# for row in range(dataset.shape[0]):
#     dataset["text"][row] = regex_symb.sub(' ',  dataset["text"][row]) # убираем дополнительные символы
#     dataset["text"][row] = dataset["text"][row].replace("\n", " ") # убираем \n
#     dataset["text"][row] = re.sub(' +', ' ', dataset["text"][row]) # оставляем максимум 1 пробел
#     dataset["text"][row] = dataset["text"][row].strip() # удаляем лишние пробелы в начале и конце статей
    

In [ ]:
# with open(r'dataset_processed.pkl', 'wb') as f:
#     pickle.dump(dataset, f)

In [3]:
with open(r'dataset_processed.pkl', 'rb') as f:
# with open(r'/content/drive/MyDrive/Универ/dataset_processed.pkl', 'rb') as f:
    dataset = pickle.load(f)

In [4]:
dataset["text"][1]

'When man had ceased to utter his lament, A god then let me tell my tale of sorrow. WHAT hope of once more meeting is there now In the still-closed blossoms of this day? Both heaven and hell thrown open seest thou; What wav ring thoughts within the bosom play No longer doubt! Descending from the sky, She lifts thee in her arms to realms on high. And thus thou into Paradise wert brought, As worthy of a pure and endless life; Nothing was left, no wish, no hope, no thought, Here was the boundary of thine inmost strife: And seeing one so fair, so glorified, The fount of yearning tears was straightway dried. No motion stirr d the day s revolving wheel, In their own front the minutes seem d to go; The evening kiss, a true and binding seal, Ne er changing till the morrow s sunlight glow. The hours resembled sisters as they went. Yet each one from another different. The last hour s kiss, so sadly sweet, effac d A beauteous network of entwining love. Now on the threshold pause the feet, now has

In [5]:
dataset.drop("text", axis=1)

,passion,daughter,weather,animal,sea,innocence,change,angel,brother,world,...,freedom,dark,hate,music,warning,teacher,peace,chicago,butterfly,murder
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13731,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
13732,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
13733,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
13734,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


Разделение на датасет для тренировки и проверки

In [6]:
train_dataset, test_dataset = train_test_split(dataset, test_size=0.2, random_state=42)

In [7]:
device = torch.device("cuda")

Topics

In [8]:
labels = dataset.columns[1:].values
labels

array(['passion', 'daughter', 'weather', 'animal', 'sea', 'innocence',
       'change', 'angel', 'brother', 'world', 'paris', 'work', 'soldier',
       'together', 'christmas', 'dance', 'sleep', 'happy', 'faith',
       'school', 'courage', 'loss', 'romance', 'nature', 'identity',
       'car', 'greed', 'lost', 'june', 'truth', 'destiny', 'trust',
       'children', 'believe', 'culture', 'silver', 'frog', 'red', 'food',
       'sun', 'romantic', 'time', 'lust', 'mother', 'heaven', 'memory',
       'crazy', 'money', 'racism', 'girl', 'home', 'evil', 'future',
       'running', 'carpe diem', 'sorrow', 'january', 'funeral', 'travel',
       'spring', 'fire', 'poem', 'suicide', 'remember', 'poetry',
       'football', 'green', 'laughter', 'cinderella', 'poverty', 'today',
       'birth', 'happiness', 'life', 'son', 'winter', 'hair', 'rose',
       'rainbow', 'god', 'star', 'house', 'power', 'sympathy',
       'depression', 'snake', 'night', 'sick', 'despair', 'husband',
       'rain', 'lov

In [9]:
id2label = {idx:label for idx, label in enumerate(labels)}
label2id = {label:idx for idx, label in enumerate(labels)}

BERT tokenizer

In [10]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased") # create a tokenizer that corresponds to BERT.

Data preprocess

In [11]:
def preprocess_data(examples):
    # take a batch of texts
    text = examples["text"]
    # encode them
    encoding = tokenizer(text, padding="max_length", truncation=True, max_length=256)
    # add labels
    labels_batch = {k: examples[k] for k in examples.keys() if k in labels}
    # create numpy array of shape (batch_size, num_labels)
    labels_matrix = np.zeros((len(text), len(labels)))
    # fill numpy array
    for idx, label in enumerate(labels):
        labels_matrix[:, idx] = labels_batch[label]

    encoding["labels"] = labels_matrix.tolist()

    return encoding

In [12]:
dd = datasets.DatasetDict({"train":Dataset.from_pandas(train_dataset),"test":Dataset.from_pandas(test_dataset)})
dd

DatasetDict({
    train: Dataset({
        features: ['text', 'passion', 'daughter', 'weather', 'animal', 'sea', 'innocence', 'change', 'angel', 'brother', 'world', 'paris', 'work', 'soldier', 'together', 'christmas', 'dance', 'sleep', 'happy', 'faith', 'school', 'courage', 'loss', 'romance', 'nature', 'identity', 'car', 'greed', 'lost', 'june', 'truth', 'destiny', 'trust', 'children', 'believe', 'culture', 'silver', 'frog', 'red', 'food', 'sun', 'romantic', 'time', 'lust', 'mother', 'heaven', 'memory', 'crazy', 'money', 'racism', 'girl', 'home', 'evil', 'future', 'running', 'carpe diem', 'sorrow', 'january', 'funeral', 'travel', 'spring', 'fire', 'poem', 'suicide', 'remember', 'poetry', 'football', 'green', 'laughter', 'cinderella', 'poverty', 'today', 'birth', 'happiness', 'life', 'son', 'winter', 'hair', 'rose', 'rainbow', 'god', 'star', 'house', 'power', 'sympathy', 'depression', 'snake', 'night', 'sick', 'despair', 'husband', 'rain', 'love', 'river', 'city', 'childhood', 'sister',

In [13]:
encoded_dataset = dd.map(preprocess_data, batched=True, remove_columns=dd['train'].column_names)

Пример

In [ ]:
# example = encoded_dataset['train'][1]
# # example = dd['train'][1]
# print(example.keys())

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels'])


In [ ]:
# example

{'input_ids': [101,
  1045,
  2156,
  1996,
  3548,
  10998,
  2006,
  4823,
  1037,
  3147,
  2162,
  2299,
  2006,
  1996,
  2346,
  2027,
  2907,
  3031,
  2037,
  4409,
  2004,
  1996,
  3554,
  3632,
  2006,
  2035,
  2154,
  2146,
  2954,
  1010,
  2233,
  1010,
  1998,
  2954,
  5607,
  1010,
  6614,
  1998,
  5607,
  2157,
  3251,
  1996,
  3103,
  12342,
  2015,
  1010,
  2023,
  2003,
  2162,
  2023,
  2003,
  2162,
  3251,
  1996,
  15811,
  10364,
  2015,
  2023,
  2003,
  2162,
  2073,
  2057,
  3102,
  2256,
  2814,
  1010,
  5208,
  1998,
  3428,
  2005,
  2027,
  2024,
  2256,
  6716,
  2005,
  2613,
  2065,
  2027,
  5454,
  2114,
  2149,
  2005,
  2178,
  2027,
  2175,
  2000,
  2162,
  2138,
  1997,
  1037,
  9210,
  1997,
  2576,
  2273,
  2040,
  3233,
  4142,
  1025,
  1999,
  14908,
  4822,
  2145,
  3173,
  2037,
  25636,
  1996,
  3548,
  2954,
  2127,
  2053,
  2028,
  4832,
  2039,
  4902,
  999,
  2954,
  1010,
  2233,
  1010,
  1998,
  2954,
  5607,
  1010,

In [ ]:
# tokenizer.decode(example['input_ids'])

'[CLS] i see the soldiers marching on singing a cold war song on the road they hold onto their guns as the fighting goes on all day long fight, march, and fight shoot, aim and shoot right whether the sun shines, this is war this is war whether the rains pours this is war where we kill our friends, sisters and brothers for they are our enemies for real if they choose against us for another they go to war because of a handful of political men who stand alive ; in erect offices still holding their pens the soldiers fight until no one stands up anymore! fight, march, and fight shoot, aim and shoot right whether the sun shines, this is war this is war whether the rains pours www. sylviachidi. com [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]

In [ ]:
# example['labels']

[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0]

In [ ]:
# [id2label[idx] for idx, label in enumerate(example['labels']) if label == 1.0]

['war']

Изменение формата

In [14]:
encoded_dataset.set_format("torch")

# Model

In [15]:
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", 
                                                           problem_type="multi_label_classification", 
                                                           num_labels=len(labels),
                                                           id2label=id2label,
                                                           label2id=label2id)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

## Train the model!

We are going to train the model using HuggingFace's Trainer API. This requires us to define 2 things: 

* `TrainingArguments`, which specify training hyperparameters. All options can be found in the [docs](https://huggingface.co/transformers/main_classes/trainer.html#trainingarguments). Below, we for example specify that we want to evaluate after every epoch of training, we would like to save the model every epoch, we set the learning rate, the batch size to use for training/evaluation, how many epochs to train for, and so on.
* a `Trainer` object (docs can be found [here](https://huggingface.co/transformers/main_classes/trainer.html#id1)).

В качестве метрики для оценивания будем использовать F1 score, так как ...

In [16]:
batch_size = 8 # размер батча

In [ ]:
# def compute_loss(self, model, inputs, return_outputs=False):
#     """
#     How the loss is computed by Trainer. By default, all models return the loss in the first element.
#     Subclass and override for custom behavior.
#     """
#     if self.label_smoother is not None and "labels" in inputs:
#         labels = inputs.pop("labels")
#     else:
#         labels = None
#     outputs = model(**inputs)
#     # Save past state if it exists
#     # TODO: this needs to be fixed and made cleaner later.
#     if self.args.past_index >= 0:
#         self._past = outputs[self.args.past_index]

#     if labels is not None:
#         if unwrap_model(model)._get_name() in MODEL_FOR_CAUSAL_LM_MAPPING_NAMES.values():
#             loss = self.label_smoother(outputs, labels, shift_labels=True)
#         else:
#             loss = self.label_smoother(outputs, labels)
#     else:
#         if isinstance(outputs, dict) and "loss" not in outputs:
#             raise ValueError(
#                 "The model did not return a loss from the inputs, only the following keys: "
#                 f"{','.join(outputs.keys())}. For reference, the inputs it received are {','.join(inputs.keys())}."
#             )
#         # We don't use .loss here since the model may return tuples instead of ModelOutput.
#         loss = outputs["loss"] if isinstance(outputs, dict) else outputs[0]

#     return (loss, outputs) if return_outputs else loss

In [17]:
class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
#         print("inputs:", inputs)
    #     labels = inputs.get("labels")
        labels = inputs.pop("labels")

        outputs = model(**inputs)
#         print("outputs:", outputs)

        # Save past state if it exists
        if self.args.past_index >= 0:
            self._past = outputs[self.args.past_index]

        logits = outputs.get("logits")

        # приводим предсказания к виду от 0 до 1
        sigmoid = torch.nn.Sigmoid()
        predictions = sigmoid(logits)

        # compute custom loss
        weights = []
        for batch in range(len(inputs["input_ids"])):
            weights.append([(1 + x*99) for x in labels[batch]])
    #     loss_fct = nn.CrossEntropyLoss(weight=torch.tensor([1.0, 2.0, 3.0]))
        loss_fct = torch.nn.BCELoss(weight=torch.tensor(weights).to(device))
        loss = loss_fct(predictions, labels)
        return (loss, outputs) if return_outputs else loss

In [127]:
loss_fct = torch.nn.BCELoss()
# input = torch.randn(batch_size, 144, requires_grad=True)
input = torch.randint(0, 2, size=(batch_size, 144), dtype=torch.float, requires_grad=True)
print(input)
# target = torch.empty(3, dtype=torch.long).random_(5)
target = torch.randint(0, 2, size=(batch_size, 144), dtype=torch.float)
print(target)
output = loss_fct(input, target)
output
# output = loss_fct(input.view(-1, len(target[0])), labels.view(-1))

tensor([[1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 0., 1.,
         1., 1., 0., 0., 0., 1., 1., 1., 0., 0., 1., 0., 1., 0., 0., 0., 1., 0.,
         1., 1., 1., 0., 0., 0., 1., 0., 0., 1., 1., 1., 0., 0., 1., 0., 1., 0.,
         1., 0., 1., 1., 1., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0.,
         1., 0., 1., 0., 1., 0., 1., 1., 1., 0., 1., 0., 0., 0., 0., 0., 1., 0.,
         0., 0., 0., 1., 1., 1., 0., 0., 0., 1., 0., 1., 0., 0., 1., 0., 1., 0.,
         1., 0., 0., 0., 0., 0., 1., 1., 0., 1., 1., 1., 1., 0., 1., 1., 0., 0.,
         0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 1., 0., 0., 1., 1., 1., 1.],
        [1., 1., 0., 0., 0., 0., 0., 1., 1., 0., 1., 0., 1., 0., 0., 0., 1., 1.,
         0., 1., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 1., 1.,
         1., 1., 1., 1., 0., 0., 1., 0., 1., 0., 1., 1., 0., 1., 0., 1., 1., 0.,
         1., 0., 0., 0., 0., 0., 1., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 0.,
         1., 0., 0., 1., 1.

tensor(50.9549, grad_fn=<BinaryCrossEntropyBackward0>)

In [18]:
args = TrainingArguments(
    f"bert-finetuned-sem_eval-english", # папка для сохранения контрольных точек
    # evaluation_strategy = "epoch", # оценивание во время обучения
    evaluation_strategy = "no",
    save_strategy = "epoch", # сохранение модели (после эпохи)
    learning_rate=2e-5, # lr для AdamW оптимизатора
    per_device_train_batch_size=batch_size, # размер батча при тренирвке
    per_device_eval_batch_size=batch_size, # размер батча при оценивании
    num_train_epochs=5, # число эпох обучения
    weight_decay=0.01, # вес для слоёв
    # load_best_model_at_end=True, # загружать ли лучшую модель
    metric_for_best_model="f1", # метрика, на основе которой выберется лучшая модель
    dataloader_drop_last=True
)

We are also going to compute metrics while training. For this, we need to define a compute_metrics function, that returns a dictionary with the desired metric values.

In [24]:
def multi_label_metrics(predictions, labels, threshold=0.5):
    # first, apply sigmoid on predictions which are of shape (batch_size, num_labels)
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions)) # переводим вероятности в область от 0 до 1

    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1 # если вероятность > treshold, то ставим 1 в предсказании (остальное 0)

    y_true = labels # реальное значение топиков
    
#     print("probs:", probs, "shape:", probs.shape)
#     print("y_true:", y_true, "shape:", y_true.shape)
#     print("y_pred:", y_pred, "shape:", y_pred.shape)
    
    f1_micro_average, accuracy, precision, recall = 0, 0, 0, 0

    for elem in range(len(y_true)):
        f1_micro_average += f1_score(y_true[elem], y_pred[elem], average='micro')
        accuracy += accuracy_score(y_true[elem], y_pred[elem])
        precision += precision_score(y_true[elem], y_pred[elem], average='micro')
        recall += recall_score(y_true[elem], y_pred[elem], average='micro')

    metrics = {'f1': f1_micro_average/len(y_true),
               'accuracy': accuracy/len(y_true),
               'precision': precision/len(y_true),
               'recall': recall/len(y_true)
              }

    return metrics

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, 
            tuple) else p.predictions
    # print("p.predictions:", p.predictions)
#     print("preds:", preds, "shape:", preds.shape)
    
    result = multi_label_metrics(
        predictions=preds, 
        labels=p.label_ids)
    return result

In [ ]:
# encoded_dataset['train'][0]['labels'].type()

In [20]:
encoded_dataset['train']['input_ids'][1]

tensor([  101,  1045,  2156,  1996,  3548, 10998,  2006,  4823,  1037,  3147,
         2162,  2299,  2006,  1996,  2346,  2027,  2907,  3031,  2037,  4409,
         2004,  1996,  3554,  3632,  2006,  2035,  2154,  2146,  2954,  1010,
         2233,  1010,  1998,  2954,  5607,  1010,  6614,  1998,  5607,  2157,
         3251,  1996,  3103, 12342,  2015,  1010,  2023,  2003,  2162,  2023,
         2003,  2162,  3251,  1996, 15811, 10364,  2015,  2023,  2003,  2162,
         2073,  2057,  3102,  2256,  2814,  1010,  5208,  1998,  3428,  2005,
         2027,  2024,  2256,  6716,  2005,  2613,  2065,  2027,  5454,  2114,
         2149,  2005,  2178,  2027,  2175,  2000,  2162,  2138,  1997,  1037,
         9210,  1997,  2576,  2273,  2040,  3233,  4142,  1025,  1999, 14908,
         4822,  2145,  3173,  2037, 25636,  1996,  3548,  2954,  2127,  2053,
         2028,  4832,  2039,  4902,   999,  2954,  1010,  2233,  1010,  1998,
         2954,  5607,  1010,  6614,  1998,  5607,  2157,  3251, 

# Training

In [21]:
trainer = CustomTrainer(
    model.to(device), # модель для обучения
    args, # аргументы для обучения
    train_dataset=encoded_dataset["train"], # датасет, используемый при обучении
    eval_dataset=encoded_dataset["test"], # датасет, используемый для оценивания
    tokenizer=tokenizer, # токенизатор
    compute_metrics=compute_metrics # функция для подсчёта метрик при оценивании
)
# trainer.compute_loss = CustomTrainer.compute_loss

In [22]:
trainer.train()

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
500,1.168100
1000,1.022500
1500,0.903000
2000,0.790000
2500,0.752300
3000,0.676000
3500,0.614300
4000,0.601000
4500,0.535300
5000,0.511900


TrainOutput(global_step=6865, training_loss=0.6771779748704081, metrics={'train_runtime': 5731.9942, 'train_samples_per_second': 9.585, 'train_steps_per_second': 1.198, 'total_flos': 7234241208975360.0, 'train_loss': 0.6771779748704081, 'epoch': 5.0})

# Оценивание (вывод метрик)

In [23]:
trainer.evaluate()

preds: [[-1.2310243  -1.9979321  -1.5411117  ... -0.8598381  -1.4206996
  -2.070729  ]
 [-1.9670153  -1.7679657  -1.3987333  ... -1.5445392  -1.2855158
  -1.4690125 ]
 [-0.3398318  -1.9901093  -1.7126558  ... -1.938248   -1.5905039
  -2.271229  ]
 ...
 [-1.4018679  -1.3828348  -1.7647464  ... -2.2269423  -1.8902483
  -2.6120212 ]
 [ 0.3692231  -2.3545976  -1.6375611  ... -2.7362573  -1.2249503
  -2.8878076 ]
 [-2.598549   -0.26765415 -2.538007   ... -2.085867   -1.5668805
  -2.3117952 ]] shape: (2744, 144)
probs: 

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



{'eval_loss': 0.6267995238304138,
 'eval_f1': 0.9547168569808938,
 'eval_accuracy': 0.9547168569808938,
 'eval_precision': 0.9547168569808938,
 'eval_recall': 0.9547168569808938,
 'eval_runtime': 188.6415,
 'eval_samples_per_second': 14.567,
 'eval_steps_per_second': 1.824,
 'epoch': 5.0}

# Проверка работоспособности

In [25]:
dataset["text"][0]

'Your passion is permanently etched on my soul, On all flesh, branding skin north and south. Even after I have finished brushing my teeth I can relive your taste in my mouth. Your poems play new chords on the strings of my heart, The mark your smile leaves? I m still guessing! But the magical way you ve touched me from the start Poetry has no way of expressing. Though it s true that we have never met in the flesh And no photograph I ve ever seen. Could it be a connection from many past lives? Just the thought of you makes my world green. The best artist could never capture the beauty Of the truth you have let my heart see, Only one thing could possibly soften this blow: To know that in fact - you are for me.'

In [47]:
# text = "I'm happy I can finally train a model for multi-label classification"
# text = dataset["text"][0]

encoding = tokenizer(text, return_tensors="pt")
encoding = {k: v.to(trainer.model.device) for k,v in encoding.items()}
outputs = trainer.model(**encoding)
outputs
logits = outputs.logits

The logits that come out of the model are of shape (batch_size, num_labels). As we are only forwarding a single sentence through the model, the batch_size equals 1. The logits is a tensor that contains the (unnormalized) scores for every individual label.

In [48]:
logits

tensor([[ 2.0125, -2.6858, -1.9812, -2.2554, -1.1505, -1.4728, -1.5059, -0.6554,
         -2.6266, -2.2363, -1.9685, -1.8338, -2.9821, -0.6778, -2.7995, -0.8191,
         -2.4721, -1.3318, -1.1549, -2.5892, -1.7923, -1.6539,  0.1912, -1.2883,
         -1.8074, -2.2431, -2.0704, -1.6269, -2.2244, -2.3247, -1.3575, -1.6078,
         -1.9353, -1.1587, -2.4454, -1.2433, -1.6355, -1.8613, -1.4203, -0.7668,
          1.0196, -0.5276,  0.9849, -3.0060, -2.6631, -1.3024, -0.7974, -1.9525,
         -2.8656, -1.7323, -2.1210, -1.9895, -0.7476, -2.0817, -0.6553, -1.3069,
         -1.7100, -2.4751, -1.4971, -1.7185, -0.3624, -1.6605, -1.6107, -0.5450,
         -2.1834, -2.3198, -0.7523, -2.7996, -1.3540, -3.0162, -1.0354, -1.0885,
         -1.6094, -1.7715, -2.7131, -1.8822, -1.6530, -1.4497, -1.5867, -2.7716,
         -1.7780, -2.2142, -1.6128, -1.2136, -2.8763, -2.3000, -1.9080, -2.8260,
         -1.5128, -1.3573, -1.9780,  0.6970, -1.1172, -2.0088, -2.1474, -2.8956,
         -2.0832,  1.0918, -

To turn them into actual predicted labels, we first apply a sigmoid function independently to every score, such that every score is turned into a number between 0 and 1, that can be interpreted as a "probability" for how certain the model is that a given class belongs to the input text.

Next, we use a threshold (typically, 0.5) to turn every probability into either a 1 (which means, we predict the label for the given example) or a 0 (which means, we don't predict the label for the given example).

In [46]:
# apply sigmoid + threshold
sigmoid = torch.nn.Sigmoid()
probs = sigmoid(logits.squeeze().cpu())
print(probs)
predictions = np.zeros(probs.shape)
predictions[np.where(probs >= 0.5)] = 1
# turn predicted id's into actual label names
predicted_labels = [id2label[idx] for idx, label in enumerate(predictions) if label == 1.0]
print(predicted_labels)

tensor([0.9008, 0.0692, 0.1084, 0.1010, 0.2075, 0.2378, 0.1810, 0.3911, 0.0631,
        0.1057, 0.1251, 0.1526, 0.0485, 0.3360, 0.0626, 0.3442, 0.0829, 0.2118,
        0.2941, 0.0674, 0.1621, 0.1569, 0.6107, 0.2437, 0.1587, 0.0912, 0.1011,
        0.1848, 0.0938, 0.1001, 0.2655, 0.1971, 0.1314, 0.2647, 0.0647, 0.2231,
        0.1675, 0.1305, 0.1838, 0.2896, 0.7905, 0.3563, 0.7507, 0.0502, 0.0674,
        0.2273, 0.3738, 0.1390, 0.0509, 0.1761, 0.1136, 0.1488, 0.3576, 0.1341,
        0.3500, 0.2302, 0.1389, 0.0708, 0.1831, 0.1541, 0.4197, 0.1604, 0.1655,
        0.3697, 0.1071, 0.0828, 0.3299, 0.0736, 0.2592, 0.0509, 0.2512, 0.2657,
        0.1859, 0.1767, 0.0707, 0.1319, 0.1748, 0.2141, 0.1493, 0.0670, 0.1363,
        0.1102, 0.1748, 0.2424, 0.0634, 0.0898, 0.1130, 0.0596, 0.2036, 0.2354,
        0.0998, 0.7391, 0.2177, 0.1124, 0.1065, 0.0591, 0.1106, 0.7406, 0.1114,
        0.4375, 0.3383, 0.0394, 0.2587, 0.1655, 0.3399, 0.1289, 0.1236, 0.0330,
        0.4882, 0.2887, 0.2213, 0.2353, 